首先，我们回顾一下多层感知机

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1586,  0.1161, -0.1529, -0.2617,  0.1806, -0.1712,  0.0619,  0.1187,
          0.1115, -0.2267],
        [-0.1393,  0.0495, -0.0175, -0.3081,  0.1623, -0.2207,  0.0959,  0.1289,
          0.2379, -0.1523]], grad_fn=<AddmmBackward>)

nn.Sequential定义了一种特殊的Module

自定义块

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [6]:
net = MLP()

X = torch.rand(2, 20)
net(X)

tensor([[ 0.1749,  0.0775,  0.0541,  0.1280,  0.0248,  0.1431,  0.2325, -0.0626,
          0.0456,  0.1636],
        [ 0.0820,  0.0592, -0.0715,  0.0599,  0.0158,  0.1148,  0.2146, -0.1600,
          0.0607,  0.0856]], grad_fn=<AddmmBackward>)

顺序块

In [11]:
class Mysequential(nn.Module):
    def __init__(self, *args):
        super().__init__() # 继承了父类的__init__()
        for block in args:
            self._modules[block] = block # 这一步是把传入的python列表，转换为_modules[block]
        
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X
    
net = Mysequential(nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10)))
net(X)

tensor([[ 0.2175, -0.1389, -0.0631, -0.0882, -0.2749,  0.0951, -0.0570, -0.0526,
          0.2251, -0.1787],
        [ 0.1712, -0.1097, -0.0040,  0.0107, -0.2449,  0.0425, -0.0582, -0.0997,
          0.2224,  0.0993]], grad_fn=<AddmmBackward>)

在正向传播函数中执行代码

In [12]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.2824, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [19]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10), nn.ReLU())
        self.linear = nn.Linear(10, 4)
    
    def forward(self, X):
        hidden = self.net(X)
        out = self.linear(hidden)
        return out
    
chimera = nn.Sequential(NestMLP(), nn.Linear(4, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0760, grad_fn=<SumBackward0>)